In [1]:
import ast
from functools import partial
import networkx as nx
import numpy as np
from typing import Dict
import matplotlib.pyplot as plt
from dataclasses import dataclass
from mpi4py import MPI
from repast4py import context as ctx
from repast4py import core, random, schedule, logging, parameters
from repast4py.network import write_network, read_network

In [6]:
# Define the number of agents (nodes)
num_agents = 1000

# Define the probability for edge creation (choose a value between 0 and 1)
# Note: Adjust p to get the desired graph density.
p = 0.01

# Create the random graph
G1 = nx.erdos_renyi_graph(n=num_agents, p=p)
G2 = nx.erdos_renyi_graph(n=num_agents, p=p)
path1, path2 = 'networks/layer1.txt', 'networks/layer2.txt'
write_network(G1, 'rumor_network', path1, 1)
write_network(G2, 'rumor_network', path2, 1)
# G1 = nx.erdos_renyi_graph(n=num_agents//2, p=p)
# G2 = nx.erdos_renyi_graph(n=num_agents//2, p=p)
# write_network(G1, 'rumor_network', path1, 1)
# write_network(G2, 'rumor_network', path2, 2)

In [7]:
# Create first network: 20 nodes with last 10 interconnected
G1 = nx.Graph()
G1.add_nodes_from(range(num_agents))

# Interconnect last 10 nodes (10-19)
edges_layer1 = [(i, j) for i in range(10, 20) for j in range(i + 1, 20)]
G1.add_edges_from(edges_layer1)

# Create second network: Erdős-Rényi graph with first 10 nodes
G2 = nx.erdos_renyi_graph(n=num_agents // 2, p=p)
write_network(G1, 'rumor_network', path1, 1)
write_network(G2, 'rumor_network', path2, 2)

In [8]:
from network import *
parse_and_write_network_files([path1, path2])

Updated network data has been written to: networks/layer1.txt_multi


In [9]:
# Parse network files to dictionaries
edges = parse_to_dictionaries([path1, path2]) 
# Compress dictionaries to string (Dictionaries -> String -> Compressed String)
compressed_edges = compress_dictionaries_to_string(edges)
# Decompress compressed string to dictionaries
decompressed_edges = decompress_string_to_dictionaries(compressed_edges) # Decompressed list of dictionaries
print(edges == decompressed_edges)

True


In [15]:
model = None

class RumorAgent(core.Agent):

    def __init__(self, nid: int, agent_type: int, rank: int, received_rumor=False, shadow=None):
        super().__init__(nid, agent_type, rank)
        self.received_rumor = received_rumor
        self.shadow = shadow or {}

    def save(self):
        """Saves the state of this agent as tuple.

        A non-ghost agent will save its state using this
        method, and any ghost agents of this agent will
        be updated with that data (self.received_rumor).

        Returns:
            The agent's state
        """
        return (self.uid, self.received_rumor, self.shadow)

    def update(self, data: bool):
        """Updates the state of this agent when it is a ghost
        agent on some rank other than its local one.

        Args:
            data: the new agent state (received_rumor)
        """
        received_rumor, shadow_data = data[0], data[1]

        if not self.received_rumor and received_rumor:
            # only update if the received rumor state
            # has changed from false to true
            model.rumor_spreaders.append(self)
            self.received_rumor = received_rumor
        
        self.shadow = shadow_data


def create_rumor_agent(nid, agent_type, rank, **kwargs):
    shadow_data = {}
    if 'data' in kwargs:  # New compressed format
        shadow_data = decompress_and_convert_shadow_data(kwargs['data'])
    return RumorAgent(nid, agent_type, rank, received_rumor=None, shadow=shadow_data)


def restore_agent(agent_data):
    uid = agent_data[0]
    received_rumor = agent_data[1]
    shadow = agent_data[2] if len(agent_data) > 2 else {}  # Handle cases where shadow might not be saved
    return RumorAgent(uid[0], uid[1], uid[2], received_rumor, shadow)

In [16]:
@dataclass
class RumorCounts:
    total_rumor_spreaders: int
    new_rumor_spreaders: int

In [17]:
class Model:
    def __init__(self, comm, params):
        self.runner = schedule.init_schedule_runner(comm)
        self.runner.schedule_stop(params['stop.at'])
        self.runner.schedule_end_event(self.at_end)

        fpath = params['network_file']
        self.context = ctx.SharedContext(comm)
        read_network(fpath, self.context, create_rumor_agent, restore_agent)
        self.net = self.context.get_projection('rumor_network')

        self.rumor_spreaders = []
        self.rank = comm.Get_rank()
        self._seed_rumor(params['initial_rumor_count'], comm)

        rumored_count = len(self.rumor_spreaders)
        self.counts = RumorCounts(rumored_count, rumored_count)
        loggers = logging.create_loggers(self.counts, op=MPI.SUM, rank=self.rank)
        self.data_set = logging.ReducingDataSet(loggers, comm, params['counts_file'])
        self.data_set.log(0)

        self.rumor_prob = params['rumor_probability']

        # Schedule layer-specific steps
        layer_schedules = params['layer_schedules']
        for layer_id, schedule_config in enumerate(layer_schedules):
            start = schedule_config['start']
            interval = schedule_config['interval']
            self.runner.schedule_repeating_event(start, interval, partial(self.new_step, layer=layer_id))
        
        for agent in self.context.agents():
           print(agent.uid, agent.shadow)
            
    def _seed_rumor(self, init_rumor_count: int, comm):
        world_size = comm.Get_size()
        rumor_counts = np.zeros(world_size, dtype=np.int32)
        if self.rank == 0:
            rng = np.random.default_rng()
            for _ in range(init_rumor_count):
                idx = rng.integers(0, high=world_size)
                rumor_counts[idx] += 1

        rumor_count = np.empty(1, dtype=np.int32)
        comm.Scatter(rumor_counts, rumor_count, root=0)

        for agent in self.context.agents(count=rumor_count[0], shuffle=True):
            agent.received_rumor = True
            self.rumor_spreaders.append(agent)

    def at_end(self):
        self.data_set.close()

    def new_step(self, layer):
        new_rumor_spreaders = []
        rng = np.random.default_rng()
        for agent in self.rumor_spreaders:
            # Agent does not have outgoing edges in this layer
            if layer not in agent.shadow.keys():
                continue
            ngh_tuples = agent.shadow[layer].keys()
            for ngh_tuple in ngh_tuples:
                ngh_agent = self.context.agent(ngh_tuple)
                if ngh_agent is None:
                    continue  # Neighbor not found (shouldn't happen if network is correct)
                # Only spread to local agents that haven't received the rumor
                if ngh_agent.local_rank == self.rank and not ngh_agent.received_rumor:
                    if rng.uniform() <= self.rumor_prob:
                        ngh_agent.received_rumor = True
                        new_rumor_spreaders.append(ngh_agent)
        # Update the list of rumor spreaders with new local agents
        self.rumor_spreaders += new_rumor_spreaders
        # Update counts
        self.counts.new_rumor_spreaders = len(new_rumor_spreaders)
        self.counts.total_rumor_spreaders += self.counts.new_rumor_spreaders
        self.data_set.log(self.runner.schedule.tick)
        # Synchronize agents across ranks
        self.context.synchronize(restore_agent)

    def start(self):
        self.runner.execute()


In [18]:
def run(params: Dict):
    global model
    model = Model(MPI.COMM_WORLD, params)
    model.start()


if __name__ == "__main__":
    params = {
        'layer_schedules': [
            {'start': 0, 'interval': 1},
            {'start': 0, 'interval': 2}
        ],
        'network_file': 'networks/layer1.txt_multi',
        'initial_rumor_count': 15,
        'stop.at': 100,
        'rumor_probability': 1,
        'counts_file': 'output/rumor_counts.csv'
    }
    run(params)

(0, 0, 0) {0: {}, 1: {}}
(1, 0, 0) {0: {}, 1: {}}
(2, 0, 0) {0: {}, 1: {}}
(3, 0, 0) {0: {}, 1: {}}
(4, 0, 0) {0: {}, 1: {}}
(5, 0, 0) {0: {}, 1: {}}
(6, 0, 0) {0: {}, 1: {}}
(7, 0, 0) {0: {}, 1: {}}
(8, 0, 0) {0: {}, 1: {}}
(9, 0, 0) {0: {}, 1: {}}
(10, 0, 0) {0: {(11, 0, 0): 1.0, (12, 0, 0): 1.0, (13, 0, 0): 1.0, (14, 0, 0): 1.0, (15, 0, 0): 1.0, (16, 0, 0): 1.0, (17, 0, 0): 1.0, (18, 0, 0): 1.0, (19, 0, 0): 1.0}, 1: {}}
(11, 0, 0) {0: {(10, 0, 0): 1.0, (12, 0, 0): 1.0, (13, 0, 0): 1.0, (14, 0, 0): 1.0, (15, 0, 0): 1.0, (16, 0, 0): 1.0, (17, 0, 0): 1.0, (18, 0, 0): 1.0, (19, 0, 0): 1.0}, 1: {}}
(12, 0, 0) {0: {(10, 0, 0): 1.0, (11, 0, 0): 1.0, (13, 0, 0): 1.0, (14, 0, 0): 1.0, (15, 0, 0): 1.0, (16, 0, 0): 1.0, (17, 0, 0): 1.0, (18, 0, 0): 1.0, (19, 0, 0): 1.0}, 1: {}}
(13, 0, 0) {0: {(10, 0, 0): 1.0, (11, 0, 0): 1.0, (12, 0, 0): 1.0, (14, 0, 0): 1.0, (15, 0, 0): 1.0, (16, 0, 0): 1.0, (17, 0, 0): 1.0, (18, 0, 0): 1.0, (19, 0, 0): 1.0}, 1: {}}
(14, 0, 0) {0: {(10, 0, 0): 1.0, (11, 0, 0